In [ ]:
# check that imports work
import rclpy
from audio_stack import beam_former

In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

import sys
sys.path.append("../src")

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False
rcParams["font.family"] = 'DejaVu Sans'
rcParams["font.size"] = 14
rcParams["text.usetex"] = True

# 0. Setup

In [ ]:
from utils.simulation import get_setup
from utils.plotting_tools import save_fig, FIGSIZE
from utils.geometry import Context
from utils.constants import PLATFORM


if PLATFORM == "crazyflie":
    from crazyflie_description_py.experiments import WALL_ANGLE_DEG_STEPPER
else:
    from epuck_description_py.experiments import WALL_ANGLE_DEG_STEPPER

plot_dir = 'plots/experiments'

distance = 15
#azimuth_deg = WALL_ANGLE_DEG_STEPPER #- 180
azimuth_deg = WALL_ANGLE_DEG_STEPPER #- 180

context = Context.get_platform_setup()
context.plot()

fig, ax = plt.subplots()
fig.set_size_inches(FIGSIZE, FIGSIZE)
source, mic_positions = get_setup(distance_cm=distance, azimuth_deg=azimuth_deg)
[ax.scatter(*mic[:2], label=f'mic{i}, $\\theta=${azimuth_deg}$^\\circ$') for i, mic in enumerate(mic_positions)]

source, mic_positions = get_setup(distance_cm=distance, azimuth_deg=azimuth_deg, ax=ax)
ax.set_title(f"wall distance $d=${distance}cm \n and angle $\\theta=${azimuth_deg} degrees")
handles, labels = ax.get_legend_handles_labels()
h_mics = {l.split(', $\\theta')[0]: h for h, l in zip(handles, labels) if '\\theta' in l}
l1 = ax.legend(h_mics.values(), h_mics.keys(),loc="lower left", bbox_to_anchor=[0.6, 0])

h_other = {l: h for h, l in zip(handles, labels) if not 'mic' in l}
ax.legend(h_other.values(), h_other.keys(),loc="lower right", bbox_to_anchor=[0.6, 0])
ax.add_artist(l1)
ax.set_xlabel("x [m]")
ax.set_ylabel("y [m]")
#save_fig(fig, f'{plot_dir}/setup_{PLATFORM}.pdf')

In [ ]:
if PLATFORM == "crazyflie":
    exp_name = "2021_07_08_stepper_fast"; # thesis
    #exp_name = "2021_10_07_stepper"
    motors = "all45000"
    bin_selection = 5
else:
    exp_name = "2021_07_27_epuck_wall"; # new sweep, better pipeline
    motors = "sweep_and_move" 
    bin_selection = 0
mic_type = "audio_deck"

In [ ]:
from utils.data_collector import DataCollector

data_collector = DataCollector()
backup_exists = data_collector.fill_from_backup(exp_name, mic_type, motors, bin_selection)
if not backup_exists: 
    print('generate data using script generate_df_results.')
df_matrix, df_dist, df_freq = data_collector.get_df_matrix()

# 2. Matrix analysis 


## 2.1 Calibration

In [ ]:
from utils.calibration import get_calibration_function_median
from utils.constants import PLATFORM

if PLATFORM == "epuck":
    motors_list = ["sweep_and_move"]
else:
    motors_list = ["all45000", 0]

for motors_here in motors_list:
    fig, axs = plt.subplots(1, 2, sharey=True)
    fig.set_size_inches(10, 5)
    calib_function_median, freqs = get_calibration_function_median(
        exp_name, mic_type, snr=bin_selection, ax=axs[0], motors=motors_here
    )
    axs[0].legend(loc='lower right')
    axs[0].set_title("calibration-individual")

    calib_function_median_one, freqs = get_calibration_function_median(
        exp_name, mic_type, ax=axs[1], fit_one_gain=True, snr=bin_selection, motors=motors_here
    )
    axs[1].set_title("calibration-global")
    axs[1].set_ylabel('')
    axs[1].legend().set_visible(False)
    save_fig(fig, f'{plot_dir}/{exp_name}_calibration_median_{motors_here}.pdf')

In [ ]:
from utils.plotting_tools import plot_df_matrix, save_fig, titles, add_colorbar
from utils.simulation import get_df_theory
from utils.data_collector import normalize_df_matrix

min_freq = min(df_freq) #3100 #min(df_freq)
max_freq = max(df_freq) #5000 #max(df_freq)

min_value = None
max_value = None

fname = f"{exp_name}_{motors}"
figsize = 2

methods = ['measured', 'theoretical'] #['raw', 'calibrated','theoretical']
mic_idx = 1
fig_all, axs_all = plt.subplots(1, len(methods), sharey=True)
fig_all.set_size_inches(len(methods)*figsize, figsize) 

for j, method in enumerate(methods):
    if method == "theoretical":
        df_norm = get_df_theory(df_freq, df_dist)
    elif method == "measured":
        df_norm = df_matrix

    n_mics = df_norm.shape[0]
    fig, axs = plt.subplots(1, n_mics, sharey=True)
    fig.set_size_inches(n_mics*figsize, figsize)
    axs[0].set_ylabel('frequency [Hz]')
    for i in range(n_mics):
        df_exp = df_norm[i]
        ax, im = plot_df_matrix(
            df_dist,
            df_freq,
            df_exp,
            ax=axs[i],
            min_freq=min_freq,
            max_freq=max_freq,
            vmin=min_value,
            vmax=max_value,
        )
        ax.set_title(f"mic{i} {method}")
        ax.set_xlabel('distance [cm]')
    ax, im = plot_df_matrix(
        df_dist,
        df_freq,
        df_norm[mic_idx],
        ax=axs_all[j],
        min_freq=min_freq,
        max_freq=max_freq,
        vmin=min_value,
        vmax=max_value,
    )
    ax.set_title(f"{method}")
    ax.set_xlabel('distance [cm]')
    #add_colorbar(fig, ax, im)
axs_all[0].set_ylabel('frequency [Hz]')
save_fig(fig_all, f"{plot_dir}/{fname}_matrices_mic{mic_idx}.png")

In [ ]:
from copy import deepcopy
from utils.inference import Inference
calib_function_median, freqs = get_calibration_function_median(
    exp_name, mic_type, snr=bin_selection, ax=axs[0], motors=motors_here
)

chosen_mics = [0,1,2,3]
d_estimates = {m:[] for m in chosen_mics}
plot_mic = 0
for i_d in range(df_matrix.shape[2]):
    signals_f = deepcopy(df_matrix[chosen_mics, :, i_d]) # 4 x 
    signals_f /= calib_function_median(df_freq)[chosen_mics]
    
    inf_machine = Inference()
    inf_machine.add_geometry(azimuth_deg=WALL_ANGLE_DEG_STEPPER, distance_range=[7, 100])
    inf_machine.add_data(np.abs(signals_f), df_freq, mics=chosen_mics)
    inf_machine.add_calibration_function(calib_function_median)
    
    #plt.figure()
    for i in chosen_mics:
        #dist_cm, probs, diff_cm = inf_machine.do_inference(algorithm="bayes", mic_idx=i, calibrate=False, interpolate=True)
        dist_cm, probs, diff_cm = inf_machine.do_inference(algorithm="cost", mic_idx=i, calibrate=False, interpolate=False)
        d_estimates[i].append(dist_cm[np.argmax(probs)])
        #plt.plot(dist_cm, probs, color=f"C{i}", label=f"mic{i}")
    #plt.legend()
    #plt.title(f"distance {i_d}")
    
fig, axs = plt.subplots(1, len(d_estimates.keys()), sharey=True)
for i, dist in d_estimates.items():
    axs[i].scatter(range(len(dist)), dist, color=f"C{i}")
    axs[i].set_title(f"mic{i}")
    axs[i].grid()

# 3. Frequency slices

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from plot_helpers import color, ls, units, labels

plot_df = pd.read_pickle("results/stepper_results_online_seed1_uniform_new.pkl")

plot_df = plot_df.loc[plot_df.method.str.startswith("calibrated")]

# add extra columns for plots
from generate_filtering_results import DISCRETIZATIONS
for discretization in plot_df.discretization.unique():
    step_cm, step_deg = DISCRETIZATIONS[discretization]
    distances_cm = np.arange(7, 80, step=step_cm)
    angles_deg = np.arange(360, step=step_deg)
    n_particles = len(distances_cm) * len(angles_deg) // 2
    plot_df.loc[plot_df.discretization == discretization, ["number angles", "number distances", "number particles"]] = (len(angles_deg), len(distances_cm), n_particles)
    
shifts = {
    "histogram 1": 0.05,
    "histogram 3": 0.1,
    "histogram 5": 0.15,
    "histogram": 0
}
for (algorithm), df_dis in plot_df.groupby(["algorithm"]):
    fig, ax = plt.subplots()
    fig.set_size_inches(2.7*figsize, 2.3*figsize)
    #fig.set_size_inches(4*figsize, figsize)
    ax_time = ax.twinx()
    particles_error = {}
    particles_runtime = {}
    for method, df in df_dis.groupby("method"):
        df["error"] = df["error"].abs()
        if "particle" in method:
            particle_median = df.groupby("number particles").median()
            particles_error[particle_median.index[0]] = particle_median.error.values[0]
            particles_runtime[particle_median.index[0]] = particle_median.runtime.values[0]
        elif "histogram" in method:
            color_name  = method.split(' ')[0]
            ls_name = method.split(color_name)[1].strip()
            
            label = labels[ls_name]
            #print("using linestyle", ls[ls_name])

            median = df.groupby("number angles").median()
            median.error += shifts[ls_name]

            xs = median["number particles"].values
            ax.semilogx(xs, median.error, ls=ls[ls_name], color='C0', label=label)
            ax_time.semilogx(xs, median.runtime, ls=ls[ls_name], color='C1')
        else:
            print("not plotting", method)
    # sort particles by number
    particles_error = dict(sorted(particles_error.items()))
    ax.plot(xs, list(particles_error.values()), ls=ls["particle"], color='C0', label="particle")
    particles_runtime = dict(sorted(particles_runtime.items()))
    ax_time.plot(xs, list(particles_runtime.values()), ls=ls["particle"], color='C1', label="particle")

    xticklabels = [f"{n_a:.0f}\n{n_d:.0f}\n{n_p:.0f}" for (n_a, n_d, n_p) in zip(median.index, median["number distances"], particles_error.keys())]
    ax.set_xticks(xs, xticklabels)

    #leg = ax.legend(loc='upper left', bbox_to_anchor=[1.2, 1.0])
    #leg = ax.legend(loc='lower center',ncol=2,bbox_to_anchor=[0.5, 1.0])
    leg = ax.legend(loc='upper center',ncol=1)
    ax.grid(True)
    ax.set_xlabel("$N_{\\theta}$ \n $N_d$ \n $N_p$")
    ax.xaxis.set_label_coords(-0.02, -0.025)
    ax.set_ylabel(f"median error [{units[algorithm]}]", color="C0")
    ax_time.set_ylabel("runtime [s]", color="C1")
    ax_time.yaxis.set_tick_params(colors="C1")
    ax.yaxis.set_tick_params(colors="C0")
    ax.set_title("time vs. performance")
    #ax.set_title(f"{algorithm}")
    #save_fig(fig, f"plots/experiments/{exp_name}_{algorithm.replace(' ', '_')}_time-error.pdf")

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from plot_helpers import color, ls, units, labels
from generate_filtering_results import N_PARTICLES

plot_df = pd.read_pickle("results/stepper_results.pkl")
plot_df = plot_df.loc[plot_df.method.str.startswith("calibrated")]
#print(plot_df)

for (algorithm), df_dis in plot_df.groupby(["algorithm"]):
    fig, ax = plt.subplots()
    ax_time = ax.twinx()
    particles_error = {}
    particles_runtime = {}
    
    index_sorted = df_dis.discretization.unique()
    pt = pd.pivot_table(df_dis, index="discretization", values=["error","runtime"], aggfunc=lambda x: np.quantile(np.abs(x), 0.7), sort=False)
    pt_std = pd.pivot_table(df_dis, index="discretization", values=["error","runtime"], aggfunc=lambda x: 0.5*np.std(np.abs(x)), sort=False)
    
    x_ticks = [N_PARTICLES[dis] for dis in pt.index.values]
    
    ax.errorbar(x_ticks, pt.error.values, pt_std.error.values, color='C0')
    ax_time.errorbar(x_ticks, pt.runtime.values, pt_std.runtime.values, color='C1')

    ax.grid(True)
    ax.set_xlabel("\# particles")
    ax.set_ylabel(f"median error [{units[algorithm]}]", color="C0")
    ax_time.set_ylabel("runtime [s]", color="C1")
    ax_time.yaxis.set_tick_params(colors="C1")
    ax.yaxis.set_tick_params(colors="C0")
    ax.set_title("time vs. performance")
    #ax.set_title(f"{algorithm}")
    #save_fig(fig, f"plots/experiments/{exp_name}_{algorithm.replace(' ', '_')}_time-error.pdf")

In [ ]:
from plot_helpers import add_double_legend, labels, ls

#plot_df = pd.read_pickle("results/stepper_results_online_seed1_uniform_new.pkl")
#plot_df = pd.read_pickle("results/stepper_results_online_seed1.pkl")

beamform = False

if PLATFORM == "crazyflie":
    name = "stepper"
elif PLATFORM == "epuck":
    name = "epuck"
print(name)

if beamform:
    fname = f"results/{name}_results_beamform.pkl"
else:
    fname = f"results/{name}_results.pkl"

plot_df = pd.read_pickle(fname)
print('read', fname)

print("figsize", figsize)
X_MAX = {
    "bayes angle": plot_df[plot_df.algorithm=="bayes angle"].error.max(),
    "bayes distance": 30 #plot_df[plot_df.algorithm=="bayes distance"].error.max()
}#plot_df.distance.max()
print(plot_df.discretization.unique())
for discretization, df_ in plot_df.groupby("discretization", sort=False):
    fig, axs = plt.subplots(1, 2)
    fig.set_size_inches(8, 4)
    for i, (algorithm, df_dis) in enumerate(df_.groupby("algorithm", sort=False)):
        for method, df in df_dis.groupby("method"):
            color_name  = method.split(' ')[0]
            ls_name = method.split(color_name)[1].strip()
            if "split particle" in ls_name:
                ls_name = "split particle"
            elif "particle" in ls_name:
                ls_name = "particle"

            cdf = sorted(np.abs(df.error))
            if color_name == "calibrated":
                label = labels[ls_name]
            else:
                label = None
            axs[i].plot(cdf, np.linspace(0, 1, len(cdf)), color=color[color_name], linestyle=ls[ls_name], label=label)
        
        axs[i].grid(True)
        axs[i].set_xlim(0, X_MAX[algorithm])
        #axs[i].set_title(f"{discretization} discretization, {algorithm}")
        axs[i].set_title(f"{algorithm.replace('bayes', '')} estimation")
        axs[i].set_xlabel(f"absolute error [{units[algorithm]}]")
        axs[i].set_ylabel("cdf [-]")
        axs[i].yaxis.set_label_coords(-0.15, 0.5)
    add_double_legend(axs[i])
    fig.subplots_adjust(wspace=0.3)
    if beamform: 
        save_fig(fig, f"plots/experiments/{exp_name}_{discretization.replace(' ', '_')}_cdfs_beamform.pdf")
    else:
        save_fig(fig, f"plots/experiments/{exp_name}_{discretization.replace(' ', '_')}_cdfs.pdf")

In [ ]:
fname = f"results/stepper_results.pkl"

plot_df = pd.read_pickle(fname)
print('read', fname)

X_MAX = {
    "bayes angle": 200,
    "bayes distance": 30
}

linestyles = {
    "calibrated": "-",
    "theoretical": ":",
    "fixed":"",
    "random":"",
}

fig, axs = plt.subplots(1, 2, sharey=True)
fig.set_size_inches(8, 4)
cmap = plt.get_cmap("viridis", lut=len(plot_df.discretization.unique()))
for j, (discretization, df_) in enumerate(plot_df.groupby("discretization", sort=False)):
    for i, (algorithm, df_dis) in enumerate(df_.groupby("algorithm", sort=False)):
        for method, df in df_dis.groupby("method"):
            if method in ["fixed", "random"]:
                continue
            ls_name, p, n = method.split(' ')
            cdf = sorted(np.abs(df.error))
            #axs[i].plot(cdf, np.linspace(0, 1, len(cdf)), color=color[color_name], linestyle=ls[ls_name], label=label)
            label = f"$N_p$={n}" if ls_name=="calibrated" else None
            axs[i].plot(cdf, np.linspace(0, 1, len(cdf)), color=cmap(j), linestyle=linestyles[ls_name], label=label)
        
        axs[i].grid(True)
        axs[i].set_xlim(0, X_MAX[algorithm])
        #axs[i].set_title(f"{discretization} discretization, {algorithm}")
        axs[i].set_title(f"{algorithm.replace('bayes', '')} estimation")
        axs[i].set_xlabel(f"absolute error [{units[algorithm]}]")
axs[1].legend()
axs[0].set_ylabel("cdf [-]")
#axs[i].yaxis.set_label_coords(-0.15, 0.5)
save_fig(fig, f"plots/experiments/{exp_name}_complexity_cdfs.pdf")

In [ ]:
from utils.constants import SPEED_OF_SOUND
print(freqs)
delta_f_med = np.median(np.diff(freqs))
max_d_med = SPEED_OF_SOUND / (4 * delta_f_med)
delta_f_min = np.min(np.diff(freqs))
max_d_min = SPEED_OF_SOUND / (4 * delta_f_min)
print("median:", max_d_med)
print("minimum:", max_d_min)

In [ ]:
from plot_helpers import add_double_legend
from utils.pandas_utils import filter_by_dict

def convert_to_str(row):
    chosen_mic = row.chosen_mics
    chosen_mic_str = "-".join(map(str, chosen_mic))
    row.chosen_mics = chosen_mic_str
    return row

plot_df = pd.read_pickle(f"results/{name}_results_mics_ablation.pkl")

X_MAX = {
    "bayes angle": plot_df[plot_df.algorithm=="bayes angle"].error.max(),
    "bayes distance": 30 #plot_df[plot_df.algorithm=="bayes distance"].error.max()
}

plot_df = plot_df.loc[~plot_df.method.isin(["fixed", "random"])]
plot_df = plot_df.apply(convert_to_str, axis=1)
discretization = plot_df.discretization.unique()[0]

colors = plt.get_cmap("viridis", lut=len(plot_df.chosen_mics.unique()))

fig, axs = plt.subplots(1, 2, sharey=True)
fig.set_size_inches(6, 2)

for i, (algorithm, df_dis) in enumerate(plot_df.groupby("algorithm", sort=False)):
    boxplots = {}
    for j, (chosen_mics, df) in enumerate(df_dis.groupby("chosen_mics", sort=False)):
        #just a sanity check that each distance only appears once
        if not len(df.distance) == len(df.distance.unique()):
            print(df)
            break
        boxplots[chosen_mics] = df.error
        cdf = sorted(np.abs(df.error))
        axs[i].plot(cdf, np.linspace(0, 1, len(cdf)), label=chosen_mics, color=colors(j))
        axs[i].grid(True)
    axs[i].set_xlim(0, X_MAX[algorithm])
    axs[i].set_title(f"{algorithm.replace('bayes', '')} estimation")
    axs[i].set_xlabel(f"absolute error [{units[algorithm]}]")
axs[0].set_ylabel("cdf [-]")
    
#fig_box, ax_plot = plt.subplots()
#ax_plot.boxplot(boxplots.values())
#ax_plot.set_xticklabels(boxplots.keys(), rotation=45)


fig.subplots_adjust(wspace=0.1)
axs[1].legend(loc="upper left", bbox_to_anchor=[1.0,1.0], title="used mics", ncol=2)
handles, labels = axs[i].get_legend_handles_labels()
axs[1].legend(handles[::1], labels[::1],ncol=2,fontsize=10, title="used mics", loc="lower left", bbox_to_anchor=[1.0, -0.2])
save_fig(fig, f"plots/experiments/{exp_name}_{discretization.replace(' ', '_')}_mics_cdfs.pdf")